In [1]:
#Importamos las librerias que nos van a ayudar en este proceso

import requests
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint
from lxml import html
from lxml.html import fromstring
import urllib.request
from urllib.request import urlopen
import random
import re
import lxml.html as lh



In [2]:
#Primero vamos a poner la url de la cual queremos sacar nuestro Pokédex

url='http://pokemondb.net/pokedex/all'

In [3]:
#Con esto tendremos en esta variable todas los contenidos de la página
pag_pokemon = requests.get(url)


In [4]:
#Guardaremos la info
base_a = lh.fromstring(pag_pokemon.content)


In [5]:
#Hacemos la info entendible por medio de hacer un parseo
tr_elementos = base_a.xpath('//tr')

#Checamos la longitud de la tabla para los primeros 5 elementos
[len(T) for T in tr_elementos[:5]]

[10, 10, 10, 10, 10]

In [6]:
tr_elementos = base_a.xpath('//tr')

#Creamos una lista vacia para poder traer la info
columna=[]
i=0

#Vamos a guardar los encabezados de cada columna
for t in tr_elementos[0]:
    i+=1
    nombre=t.text_content()
    print (i,nombre)
    columna.append((nombre,[]))

1 #
2 Name
3 Type
4 Total
5 HP
6 Attack
7 Defense
8 Sp. Atk
9 Sp. Def
10 Speed


In [7]:
#Toda la información tiene que guardarse en el segundo valor de la tabla ya que el primero son los encabezados
for j in range(1,len(tr_elementos)):
    T=tr_elementos[j]
    
    #Hay que delimitar que solo llegue al 10 porque si no esta no es información de nuestra tabal
    if len(T)!=10:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
       
        if i>0:
        
            try:
                data=int(data)
            except:
                pass
        
        columna[i][1].append(data)
        
        i+=1

In [8]:
#La tabla tiene más registros porque contiene todas las variantes y megaevoluciones de todos los Pokémon
[len(C) for (title,C) in columna]

[1045, 1045, 1045, 1045, 1045, 1045, 1045, 1045, 1045, 1045]

In [9]:
Dict={title:column for (title,column) in columna}
df=pd.DataFrame(Dict)

In [10]:
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,004,Charmander,Fire,309,39,52,43,60,50,65


In [11]:
#Ahora vamos a hacer la limpieza, en las columnas de nombre donde aparecen las mega evoluciones 
# o diferentes Pokémon así como la columna de tipo que contiene los 2 tipos en algunos casos

def parentesis(elemento):
    list = [x for x in elemento]
    for m in range(1, len(list)):
        if list[m].isupper():
            list[m] = list[m]
    lista_final = ''.join(list).split(' ')
    largo = len(lista_final)
    if largo>1:
        lista_final.insert(1,'(')
        lista_final.append(')')
    return ' '.join(lista_final)


In [12]:
#Ahora va para los cambios en el df

df['Name']=df['Name'].apply(parentesis)
Type = df["Type"].str.split(expand=True)
Type.columns = ['Type_1', 'Type_2']
#df.head()

df = pd.concat([df, Type], axis=1)
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type_1,Type_2
0,001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45,Grass,Poison
1,002,Ivysaur,Grass Poison,405,60,62,63,80,80,60,Grass,Poison
2,003,Venusaur,Grass Poison,525,80,82,83,100,100,80,Grass,Poison
3,003,Venusaur ( Mega Venusaur ),Grass Poison,625,80,100,123,122,120,80,Grass,Poison
4,004,Charmander,Fire,309,39,52,43,60,50,65,Fire,None


In [13]:
df.to_csv('Pokedex_Mich.csv')